# 1. Подготовка данных

In [1]:
#импорт необходимых библиотек
import pandas as pd 
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
import matplotlib.pyplot as plt
from scipy.stats import skewnorm
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.preprocessing import OrdinalEncoder
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
import time


In [2]:
#загрузка данных
data = pd.read_csv('/datasets/autos.csv')

In [3]:
#приведение название столбцов к нижнему регистру
data.columns = data.columns.str.lower()

In [4]:
#удаляем лишние столбцы из данных
data = data.drop(['datecrawled', 'registrationmonth', 'datecreated', 'numberofpictures', 'postalcode', 'lastseen'], axis=1)

In [5]:
#удаляем строки где цена меньше 200 и кол-во л/с меньше 30
data = data.drop(np.where(data['price'] < 200)[0]).reset_index()
data = data.drop(np.where(data['power'] <= 30)[0]).reset_index()

In [6]:
data = data.drop(['index', 'level_0'], axis = 1)

In [7]:
#удаляем строки где кол-во л/с больше 500
data = data.drop(np.where(data['power'] > 500)[0]).reset_index()

In [8]:
#удаляем строки где дата выпуска автомобиля раньше 1910 года
data = data.drop(np.where(data['registrationyear'] < 1910)[0]).reset_index()

In [9]:
#удаляем строки где дата выпуска автомобиля больше 2018 года
data = data.drop(np.where(data['registrationyear'] > 2018)[0])

In [10]:
data = data.drop(['index', 'level_0'], axis = 1)

In [11]:
#удаляем столбец с пробегом автомобиля
data = data.drop(['kilometer'], axis = 1)

In [12]:
display(data.head(10))

,price,vehicletype,registrationyear,gearbox,power,model,fueltype,brand,notrepaired
0,18300,coupe,2011,manual,190,NaN,gasoline,audi,yes
1,9800,suv,2004,auto,163,grand,gasoline,jeep,NaN
2,1500,small,2001,manual,75,golf,petrol,volkswagen,no
3,3600,small,2008,manual,69,fabia,gasoline,skoda,no
4,650,sedan,1995,manual,102,3er,petrol,bmw,yes
5,2200,convertible,2004,manual,109,2_reihe,petrol,peugeot,no
6,14500,bus,2014,manual,125,c_max,petrol,ford,NaN
7,999,small,1998,manual,101,golf,NaN,volkswagen,NaN
8,2000,sedan,2004,manual,105,3_reihe,petrol,mazda,no
9,2799,wagon,2005,manual,140,passat,gasoline,volkswagen,yes


In [13]:
data = data.fillna('unknown')

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 301964 entries, 0 to 301989
Data columns (total 9 columns):
price               301964 non-null int64
vehicletype         301964 non-null object
registrationyear    301964 non-null int64
gearbox             301964 non-null object
power               301964 non-null int64
model               301964 non-null object
fueltype            301964 non-null object
brand               301964 non-null object
notrepaired         301964 non-null object
dtypes: int64(3), object(6)
memory usage: 23.0+ MB


In [15]:
#проверка наличия пропусков в данных
print(data.isna().sum())

price               0
vehicletype         0
registrationyear    0
gearbox             0
power               0
model               0
fueltype            0
brand               0
notrepaired         0
dtype: int64


### Выводы. Шаг 1
1) При первичном анализе были выявлены столбцы, которые малоинформативны для обучения модели, данные столбцы были удалены: 'datecrawled', 'registrationmonth', 'datecreated', 'numberofpictures', 'postalcode', 'lastseen'   
2) При анализе данных были выявлены ошибочные показатели в некоторых столбцах, показатели типа "цена = 0, кол-во л/с больше 1000, год выпуска автомобиля 5000 год". Строки были удалены по следующим критериям: цена меньше 200 долларов, кол-во л/с меньше 30 и больше 500, год регистрации автомобиля меньше 1910 и больше 2018 года.   
3) При более детальном разборе столбца 'kilometer', выяснилось что большинство значений в столбце больше 150к километров - скорей всего столбец заполнен некоректными данными, что может в последствии повлиять на модель - было принято решение удалить данный столбец.  
4) После выполнения первых трех пунктов в таблице сохранились пропущенные значения в столбцах: 'vehicletype', 'gearbox', 'model', 'fueltype', 'notrepaired' - заполнить их, к сожалению, нечем.   

# 2. Обучение моделей

In [16]:
encoder = OrdinalEncoder()
data[["vehicletype", "gearbox", "model", "fueltype", "brand", "notrepaired"]] = encoder.fit_transform(data[["vehicletype", "gearbox", "model", "fueltype", "brand", "notrepaired"]])

In [17]:
display(data.head(10))

,price,vehicletype,registrationyear,gearbox,power,model,fueltype,brand,notrepaired
0,18300,2.0,2011,1.0,190,227.0,2.0,1.0,2.0
1,9800,6.0,2004,0.0,163,117.0,2.0,14.0,1.0
2,1500,5.0,2001,1.0,75,116.0,6.0,38.0,0.0
3,3600,5.0,2008,1.0,69,101.0,2.0,31.0,0.0
4,650,4.0,1995,1.0,102,11.0,6.0,2.0,2.0
5,2200,1.0,2004,1.0,109,8.0,6.0,25.0,0.0
6,14500,0.0,2014,1.0,125,60.0,6.0,10.0,1.0
7,999,5.0,1998,1.0,101,116.0,7.0,38.0,1.0
8,2000,4.0,2004,1.0,105,10.0,6.0,19.0,0.0
9,2799,8.0,2005,1.0,140,170.0,2.0,38.0,2.0


In [18]:
#подготовка валидационной и обучающей выборки
data_train, data_valid = train_test_split(data, test_size=0.25, random_state=12345)
features_train = data_train.drop(['price'], axis=1)
target_train = data_train['price']
features_valid = data_valid.drop(['price'], axis=1)
target_valid = data_valid['price']

In [19]:
def RMSE(y_true,y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = mse ** 0.5
    return rmse

In [20]:
#определение критерия по оценки модели
my_scorer = make_scorer(RMSE, greater_is_better=True)

In [21]:
#определение гипепараметров для модели "случайный лес"
model_for = RandomForestRegressor()

#parametrs = {'max_depth': range(1,4), 'n_estimators': range(1, 21, 5)}
#grid_search = GridSearchCV(model_for, parametrs, cv=3, scoring = my_scorer)
#grid_search.fit(features_train, target_train)
#grid_search.best_params_

In [22]:
#определение гипепараметров для модели "дерево решений"
model_tree = DecisionTreeRegressor()

#parametrs = {'max_depth': range(1,4), 'min_samples_split': range(2, 8, 2)}
#grid_search = GridSearchCV(model_tree, parametrs, cv=3, scoring = my_scorer)
#grid_search.fit(features_train, target_train)
#grid_search.best_params_

In [23]:
#определение гипепараметров для модели "LGB"
model_lgb = LGBMRegressor(boosting_type ='gbdt')

#parametrs = {'max_depth': range(1,3), 'num_leaves': range(31, 93, 31)}
#grid_search = GridSearchCV(model_lgb, parametrs, cv=3, scoring = my_scorer)
#grid_search.fit(features_train, target_train)
#grid_search.best_params_

# 3. Анализ моделей

In [24]:
start_time_for_val = time.clock()

#обучение модели "Случайный лес"
model_for = RandomForestRegressor(max_depth=3, n_estimators=16)
cross_for = cross_val_score(model_for, features_train, target_train, cv=3, scoring = my_scorer)
print('Значение RMSE для случайного леса (обучающие данные)', "%.2f" % cross_for.mean())

time_for_val = time.clock() - start_time_for_val
print ("Время обучения модели:", "%.2f" % time_for_val, "секунды")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


Значение RMSE для случайного леса (обучающие данные) 3011.39
Время обучения модели: 3.42 секунды


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


In [25]:
start_time_for_pre = time.clock()

#предсказание модели "Случайный лес"
model_for.fit(features_train, target_train)
predict_for = model_for.predict(features_valid)
print('Значение RMSE для случайного леса (валидационные данные)', "%.2f" % RMSE(predict_for, target_valid))

time_for_pre = time.clock() - start_time_for_pre
print ("Время предсказания модели:", "%.2f" % time_for_pre, "секунды")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


Значение RMSE для случайного леса (валидационные данные) 3025.28
Время предсказания модели: 1.72 секунды


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


In [26]:
start_time_tree_val = time.clock()

#обучение модели "Дерево решений"
model_tree = DecisionTreeRegressor(max_depth=1, min_samples_split=2)
cross_tree = cross_val_score(model_tree, features_train, target_train, cv=3, scoring = my_scorer)
print('Значение RMSE для дерева решений (обучающие данные)', "%.2f" % cross_tree.mean())

time_tree_val = time.clock() - start_time_tree_val
print ("Время обучения модели:","%.2f" % time_tree_val, "секунды")

Значение RMSE для дерева решений (обучающие данные) 3761.88
Время обучения модели: 0.18 секунды


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


In [27]:
start_time_tree_pre = time.clock()

#предсказание модели "Дерево решений"
model_tree.fit(features_train, target_train)
predict_tree = model_tree.predict(features_valid)
print('Значение RMSE для случайного леса (валидационные данные)', "%.2f" % RMSE(predict_tree, target_valid))

time_tree_pre = time.clock() - start_time_tree_pre
print ("Время предсказания модели:", "%.2f" % time_tree_pre, "секунды")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


Значение RMSE для случайного леса (валидационные данные) 3760.37
Время предсказания модели: 0.07 секунды


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


In [28]:
start_time_log_val = time.clock()

#обучение модели "Линейная регрессия"
model_log = LinearRegression()
cross_log = cross_val_score(model_log, features_train, target_train, cv=3, scoring = my_scorer)
print('Значение RMSE для линейной регрессии (обучающие данные)', "%.2f" % cross_log.mean())

time_log_val = time.clock() - start_time_log_val
print ("Время обучения модели:","%.2f" % time_log_val, "секунды")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


Значение RMSE для линейной регрессии (обучающие данные) 3317.65
Время обучения модели: 0.54 секунды


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


In [29]:
start_time_log_pre = time.clock()

#предсказания модели "Линейная регрессия"
model_log.fit(features_train, target_train)
predict_log = model_log.predict(features_valid)
print('Значение RMSE для линейной регрессии (валидационные данные)', "%.2f" % RMSE(predict_log, target_valid))

time_log_pre = time.clock() - start_time_log_pre
print ("Время предсказания модели:", "%.2f" % time_log_pre, "секунды")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


Значение RMSE для линейной регрессии (валидационные данные) 3334.06
Время предсказания модели: 0.17 секунды


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


In [30]:
start_time_cat_val = time.clock()

#обучение модели "CatBoost"
model_cat = CatBoostRegressor(iterations=2, depth=10, loss_function='RMSE')
cross_cat = cross_val_score(model_cat, features_train, target_train, cv=3, scoring = my_scorer)
print('Значение RMSE для catboost (обучающие данные)', "%.2f" % cross_cat.mean())

time_cat_val = time.clock() - start_time_cat_val
print ("Время обучения модели:","%.2f" % time_cat_val, "секунды")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


0:	learn: 4485.1441981	total: 218ms	remaining: 218ms
1:	learn: 4391.1591278	total: 413ms	remaining: 0us
0:	learn: 4489.6762165	total: 113ms	remaining: 113ms
1:	learn: 4395.9404419	total: 311ms	remaining: 0us
0:	learn: 4490.3281367	total: 121ms	remaining: 121ms
1:	learn: 4396.5935982	total: 309ms	remaining: 0us
Значение RMSE для catboost (обучающие данные) 4394.78
Время обучения модели: 2.11 секунды


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


In [31]:
start_time_cat_pre = time.clock()

#предсказание модели "CatBoost"
model_cat.fit(features_train, target_train)
predict_cat = model_cat.predict(features_valid)
print('Значение RMSE для catboost (валидационные данные)', "%.2f" % RMSE(predict_cat, target_valid))

time_cat_pre = time.clock() - start_time_cat_pre
print("Время предсказания модели:", "%.2f" % time_cat_pre, "секунды")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


0:	learn: 4488.6358072	total: 191ms	remaining: 191ms
1:	learn: 4394.2983003	total: 481ms	remaining: 0us
Значение RMSE для catboost (валидационные данные) 4389.82
Время предсказания модели: 0.98 секунды


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


In [32]:
start_time_lgb_val = time.clock()

#обучение модели "LGB"
model_lgb = LGBMRegressor(boosting_type ='gbdt', max_depth=1, num_leaves=31)
cross_lgb = cross_val_score(model_lgb, features_train, target_train, cv=3, scoring = my_scorer)
print('Значение RMSE для LGB (обучающие данные)', "%.2f" % cross_lgb.mean())

time_lgb_val = time.clock() - start_time_lgb_val
print ("Время обучения модели:","%.2f" % time_lgb_val, "секунды")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


Значение RMSE для LGB (обучающие данные) 2565.54
Время обучения модели: 7.50 секунды


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


In [33]:
start_time_lgb_pre = time.clock()

#предсказания модели "LGB"
model_lgb.fit(features_train, target_train)
predict_lgb = model_lgb.predict(features_valid)
print('Значение RMSE для LGB (валидационные данные)', "%.2f" % RMSE(predict_lgb, target_valid))

time_lgb_pre = time.clock() - start_time_lgb_pre
print ("Время предсказания модели:", "%.2f" % time_lgb_pre, "секунды")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


Значение RMSE для LGB (валидационные данные) 2583.74
Время предсказания модели: 3.94 секунды


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


In [34]:
#таблица с общими результатами
ml = ['Случайный лес', 'Дерево решений', 'Линейная регрессия', 'CatBoost', 'LGB']
data = {
  'Результаты RMSE': [RMSE(predict_for, target_valid), RMSE(predict_tree, target_valid), RMSE(predict_log, target_valid), RMSE(predict_cat, target_valid), RMSE(predict_lgb, target_valid)],
  'Время обучения в секундах':  [time_for_val, time_tree_val, time_log_val, time_cat_val, time_lgb_val],
  'Время предсказания в секундах':  [time_for_pre, time_tree_pre, time_log_pre, time_cat_pre, time_lgb_pre]
}
data_ml = pd.DataFrame(data=data, index=ml)
display(data_ml)

,Результаты RMSE,Время обучения в секундах,Время предсказания в секундах
Случайный лес,3025.282117,3.422768,1.722094
Дерево решений,3760.374369,0.177299,0.068503
Линейная регрессия,3334.060101,0.537838,0.167970
CatBoost,4389.816948,2.108125,0.977959
LGB,2583.738656,7.496357,3.936358


### Выводы. Шаг 3
1) Выводы получились очень даже интересные: самая быстрой моделью оказалась - Дерево решений (и по времени обучения и по времени предсказания), но по качеству она внизу. Самой качественной оказалась - LGB, но по зато по скорости она в самом низу (и по времени обучения и по времени предсказания).    
2) Получается при обучение моделью необходимо чем либо жертвовать? скорость или качеством?) - ревьюер сможете подсказать?)   
3) На шаге 2, оставил коды по подбору наилучшей модели закомментированными, так как степик ложится когда все это оставляешь). 